In [1]:
import duckdb
import pandas as pd
import json
import yaml
import openai
from tqdm import tqdm
import os
from pathlib import Path
import re


In [2]:
con = duckdb.connect("drug.db")
#con = duckdb.connect()


In [3]:
con.sql("""CREATE TABLE Trials (
            PostingID     VARCHAR NOT NULL PRIMARY KEY,
            Sponsor    VARCHAR,
            StudyTitle     VARCHAR,
            Disorder   VARCHAR,
            Phase   VARCHAR,
            LinkToSponsorStudyRegistry   VARCHAR,
            LinkToClinicalTrials   VARCHAR,
            drug_cui  VARCHAR[],
            drug_names   VARCHAR[],
            )""")

con.sql("""CREATE TABLE Drug (
            drug_cui  VARCHAR NOT NULL PRIMARY KEY,
            name  VARCHAR
            )""")

con.sql("""CREATE TABLE Disorder (
            disorder_cui  VARCHAR NOT NULL PRIMARY KEY,
            name  VARCHAR,
            definition  VARCHAR
            )""")

con.sql("""CREATE TABLE MOA (
            moa_id  VARCHAR NOT NULL PRIMARY KEY,
            name  VARCHAR
            )""")

con.sql("""CREATE TABLE DrugDisorder (
            drug_cui  VARCHAR NOT NULL REFERENCES Drug(drug_cui),
            disorder_cui  VARCHAR NOT NULL REFERENCES Disorder(disorder_cui)
            )""")

con.sql("""CREATE TABLE DrugMOA (
            drug_cui  VARCHAR NOT NULL REFERENCES Drug(drug_cui),
            moa_id  VARCHAR NOT NULL REFERENCES MOA(moa_id)
            )""")




In [4]:
df = pd.read_csv("drugs/trials_umls.tsv", sep='\t', lineterminator='\n')

con.sql("""BEGIN TRANSACTION;""")


for index, row in df.iterrows():
  con.execute("""
                INSERT INTO Trials (
                    PostingID,
                    Sponsor,
                    StudyTitle,
                    Disorder,
                    Phase,
                    LinkToSponsorStudyRegistry,
                    LinkToClinicalTrials,
                    drug_cui,
                    drug_names
                ) 
                SELECT 
                    $1,
                    $2,
                    $3,
                    $4,
                    $5,
                    $6,
                    $7,
                    $8::VARCHAR[],
                    $9::VARCHAR[]
            """, [
                str(row["Posting ID"]),
                row["Sponsor"],
                row["Study Title"],
                row["Medical Condition"],
                row["Phase"],
                row["Link to Sponsor Study Registry"],
                row["Link to study details on ClinicalTrials.gov (if available)"],
                row["CUI_umls"].split(";"),
                row["preferred_umls_name"].split(";")
            ])

con.sql("""PRAGMA create_fts_index('Trials', 'PostingID', 'StudyTitle', overwrite=1);""")

con.sql("""COMMIT;""")



In [5]:
con.sql("""SELECT StudyTitle FROM Trials LIMIT 5;""")

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                                                      StudyTitle                                                                                                                      │
│                                                                                                                       varchar                                                                                                                        │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ U0

In [6]:
db = "Trials"

query = f"""SELECT PostingID as trial_id, StudyTitle, score
FROM (
    SELECT *, fts_main_{db}.match_bm25(
        PostingID,
        'double blind & Valaciclovir',
        fields := 'StudyTitle'
    ) AS score
    FROM {db}
)
WHERE score IS NOT NULL
ORDER BY score DESC LIMIT 5;
"""

con.sql(query)

┌──────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────┐
│ trial_id │                                                                                                      StudyTitle                                                                                                       │       score        │
│ varchar  │                                                                                                        varchar                                                                                                        │       double       │
├──────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼────────────────────┤


In [7]:
con.sql("""BEGIN TRANSACTION;""")

for line in open("drugs/drugs_8.json", "r"):
  value = json.loads(line)

  con.execute("""
                INSERT INTO Drug (
                    drug_cui,
                    name
                ) 
                SELECT 
                    $1,
                    $2
            """, [
                value["cui"],
                value["name"]
            ])

  #query = f"""INSERT INTO Drug VALUES ('{value["cui"]}', '{value["name"].replace("'", "''")}');"""
  #print(query)
  #con.sql(query)

con.sql("""COMMIT;""")

In [8]:
con.sql("""
SELECT * FROM Drug WHERE drug_cui = 'C0046056'
""")

┌──────────┬────────────────────────┐
│ drug_cui │          name          │
│ varchar  │        varchar         │
├──────────┼────────────────────────┤
│ C0046056 │ fluorodeoxyglucose F18 │
└──────────┴────────────────────────┘

In [9]:
con.sql("""BEGIN TRANSACTION;""")

for line in open("drugs/disease_8_embedding.json", "r"):
  value = json.loads(line)

  con.execute("""
                INSERT INTO Disorder (
                    disorder_cui,
                    name,
                    definition
                    
                ) 
                SELECT 
                    $1,
                    $2,
                    $3
            """, [
                value["cui"],
                value["name"],
                value["definition"]
            ])

  #query = f"""INSERT INTO Disorder VALUES ('{value["cui"]}', '{value["name"].replace("'", "''")}', '{value["definition"].replace("'", "''")}', {value["definitionEmbedding"]});"""
  #print(query)
  #con.sql(query)

con.sql("""COMMIT;""")

In [10]:
con.sql("""BEGIN TRANSACTION;""")

df = pd.read_csv("drugs/moa_8.tsv", sep='\t', lineterminator='\n')

for index, row in df.iterrows():

    con.execute("""
                INSERT INTO MOA (
                    moa_id,
                    name
                ) 
                SELECT 
                    $1,
                    $2
            """, [
                row["id"],
                row["name"]
            ])

  #query = f"""INSERT INTO MOA VALUES ('{row["id"]}', '{row["name"].replace("'", "''")}');"""
  #print(query)
  #con.sql(query)


con.sql("""COMMIT;""")

In [11]:
con.sql("""BEGIN TRANSACTION;""")
df = pd.read_csv("drugs/cui_indication_8.tsv", sep='\t', lineterminator='\n')

for index, row in df.iterrows():
    con.execute("""
              INSERT INTO DrugDisorder (
                  drug_cui,
                  disorder_cui
              ) 
              SELECT 
                  $1,
                  $2
          """, [
              row["from"],
              row["to"]
          ])
  #query = f"""INSERT INTO DrugDisorder VALUES ('{row["from"]}', '{row["to"]}');"""
  #con.sql(query)



con.sql("""COMMIT;""")

In [12]:
con.sql("""BEGIN TRANSACTION;""")
df = pd.read_csv("drugs/cui_moa_8.tsv", sep='\t', lineterminator='\n')

for index, row in df.iterrows():
  con.execute("""
              INSERT INTO DrugMOA (
                  drug_cui,
                  moa_id
              ) 
              SELECT 
                  $1,
                  $2
          """, [
              row["from"],
              row["to"]
          ])
  

  #query = f"""INSERT INTO DrugMOA VALUES ('{row["from"]}', '{row["to"]}');"""
  #con.sql(query)


con.sql("""COMMIT;""")

1. Prescription Volume and Drug Interactions: "Which commonly prescribed medications in 2022 have the highest number of severe drug-drug interactions, and what alternative treatments with similar mechanisms of action (MOA) might present fewer interaction risks?”: A scatter chart with interaction count vs prescription, and then search for alternative?

In [13]:
con.sql("""
CREATE VIEW Trial_view AS SELECT
        *,
        list_aggregate(list_transform(drug_cui, x -> CAST(x AS VARCHAR)), 'string_agg', ';') AS drug_cui_str,
        list_aggregate(list_transform(drug_names, x -> CAST(x AS VARCHAR)), 'string_agg', ';') AS drug_names_str,

        FROM Trials
""")

In [14]:
con.sql("SELECT * FROM Trial_view LIMIT 5;")
#con.sql("SELECT * FROM Trials LIMIT 5;")


┌───────────┬─────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────┬─────────┬───────────────────────────────────────────────────────────────┬────────────────────────────────────────────┬──────────────────────┬───────────────────────────────────────────┬───────────────────┬────────────────────────────────────────┐
│ PostingID │ Sponsor │                                                                                                                      StudyTitle                                                                                                                      │           Disorder           │  Phase  │                  LinkToSponsorStudyRegistry                   │            LinkToClinicalTrials            │       drug_cui       │        

In [15]:
con.close()